<a href="https://colab.research.google.com/github/ISaySalmonYouSayYes/LLM_mit_LangChain/blob/main/LLM_Agent_Simulate_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Before you start**
- The document is [Here](https://github.com/ISaySalmonYouSayYes/LLM_mit_LangChain/blob/main/simulate_exp.md)  
- Contents is always your best friend:P

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('openAI_Key')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai
    !pip install langgraph

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_experimental langchainhub sentence-transformers

# **1. GPT-related method**  
- Using GPT4.0-mini mit Langchain(A library uniforms the syntax of all LLMs)
- Generating Persona(Characters) by LLM
  - Few-shot learning
  - Work perfectly with this prompt

In [20]:
import random

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import PromptTemplate

def GPT4_request(prompt:str):
  """
  Given a prompt, make a request to OpenAI server and returns the response.
  ARGS:
    prompt: a str prompt
  RETURNS:
    a str of GPT-4's response.
  """
  MODEL = 'gpt-4o-mini'
  messages = [
    SystemMessage(content="user"),
    HumanMessage(content= prompt),
    ]

  llm = ChatOpenAI(
    model=MODEL,
    temperature= 0.0,
    n= 1,
    max_tokens= 256)

  return llm.invoke(messages).content

def GPT4_generate_persona(name:str):
  MODEL = 'gpt-4o-mini'
  prompt = f"""
  Please creating information for {name} following the persona template. Don't add extra words.

  ----------------------------------------
  Template:
      name (str): The name of the persona.
      age (str): Age of the persona.
      gender (str): Gender of the persona.(Only Female, and Male)
      status (str): Occupation or current status.
      hobbies (list): List of hobbies the persona enjoys.
      wealth (str): Wealth level (0 to 1000).
      belongings (list): Persona's belongings.
      religious (str): Persona's religious
      ethnicity (str): Persona's ethnicity.
  ----------------------------------------
  Example:
        {
            "name": "Koichiro",
            "age": "29",
            "gender": "Male",
            "status": "Student",
            "hobbies": ["painting", "cycling", "reading sci-fi"],
            "wealth": "550",
            "belongings": ["book"],
            "religious": "Buddhism",
            "ethnicity": "Japanese"
        }


        {
            "name": "Jill",
            "age": "27",
            "gender": "Female",
            "status": "Software Developer",
            "hobbies": ["hiking", "playing video games", "cooking"],
            "wealth": "900",
            "belongings": ["water bottle", "pencil", "knife"],
            "religious": "Catholic",
            "ethnicity": "German"
        }

        {
            "name": "Midori",
            "age": "25",
            "gender": "Female",
            "status": "Graphic Designer",
            "hobbies": ["drawing", "photography", "traveling"],
            "wealth": "700",
            "belongings": ["laptop", "camera", "sketchbook"],
            "religious": "Shinto",
            "ethnicity": "Japanese"
        }


  """


  messages = [
    SystemMessage(content="You're a system creating a simulated person"),
    HumanMessage(content= prompt),
    ]

  llm = ChatOpenAI(
    model=MODEL,
    temperature= 1.2,
    n= 1,
    max_tokens= 256)

  return llm.invoke(messages).content


#Debug ---- remove content in return line to see metadata
# print(GPT4_request("Hiii, what is your name?"))

# **2. Persona.py**
- Create an object Persona documenting the personal info
- Generating Persona(Characters) by LLM
  - Few-shot learning
  - Work perfectly with this prompt

In [15]:
class Persona:
    def __init__(self, name, age, gender, status, hobbies, wealth, belongings, religious = None, ethnicity=None):
        """
        Initializes a Persona instance with a mix of essential and fun attributes.

        Args:
            name (str): The name of the persona.
            age (int): Age of the persona.
            gender (str): Gender of the persona.
            status (str): Occupation or current status.
            hobbies (list): List of hobbies the persona enjoys.
            wealth (int): Wealth level.
            belongings (list): Persona's belongings.
            religious (str): Persona's religious
            ethnicity (str): Persona's ethnicity
        """
        self.name = name
        self.age = age
        self.gender = gender
        self.status = status
        self.hobbies = hobbies
        self.wealth = wealth
        self.belongings = belongings
        self.religious = religious
        self.ethnicity = ethnicity

    def generate_persona_from_gpt(name:str):
      gpt_response = GPT4_generate_persona(name)
      persona_args = eval(f"dict({gpt_response})")
      return Persona(**persona_args)

    def display_info(self):
        """Displays basic information about the persona."""
        print(f"Name: {self.name}")
        print(f"Age: {self.age}")
        print(f"Gender: {self.gender}")
        print(f"Status: {self.status}")
        print(f"Hobbies: {self.hobbies}")
        print(f"Wealth: {self.wealth}")
        print(f"Belongings: {self.belongings}")
        print(f"Religious: {self.religious}")
        print(f"Ethnicity: {self.ethnicity}")

    def info_to_dict(self):
      """Returns a dictionary containing the persona's attributes."""
      return {
          "name": self.name,
          "age": self.age,
          "gender": self.gender,
          "status": self.status,
          "hobbies": self.hobbies,
          "wealth": self.wealth,
          "belongings": self.belongings,
          "religious": self.religious,
          "ethnicity": self.ethnicity,
      }

    def info_to_string_story(self):
      return f'I am {self.name}. My age is {self.age}. My gender is {self.gender}. My status is {self.status}. My hobbies are {self.hobbies}.\
      My wealth is {self.wealth}. My belongings is {self.belongings}. My religious is {self.religious}. My ethnicity is {self.ethnicity}'

    def update_info(self, **kwargs):
      """
      Updates the persona's attributes with new values provided as keyword arguments.

      Args:
          **kwargs: Attribute names as keys and their new values as values.
                    Supported attributes: name, age, gender, status, hobbies,
                    wealth, belongings, religious, ethnicity.
      """
      for key, value in kwargs.items():
          if hasattr(self, key):
              setattr(self, key, value)
              print(f"Updated {key} to {value}")
          else:
              print(f"Attribute {key} not found in Persona.")

# **3. Talk.py**

In [ ]:
import re
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from IPython.display import display_markdown

class Talk:
  def begin_conversation(persona_name, opposite_persona_name, persona_info, opposite_persona_info):
      MODEL = 'gpt-4o-mini'

      system_message = (
      f"You are {persona_name}. Here is your background:\n"
      f"{persona_info}\n\n"
      f"You are speaking with {opposite_persona_name}, whose background is:\n"
      f"{opposite_persona_info}\n\n"
      "Guidelines:\n"
      "1. Format answers with markdown.\n"
      "2. Answer based on the background information provided.\n"
      f"3. Repeat your name before answering (e.g., \"{persona_name}: ...\").\n"
      "4. Stick to the original topic.\n"
      "5. Avoid repeating yourself.\n"
      )


      # Create system and human message templates
      system_message_template = SystemMessagePromptTemplate.from_template(
          system_message
      )


      human_message_template = HumanMessagePromptTemplate.from_template(
          """
          Conversation: {history}
          Human: {input}
          """
      )

      # Combine into a ChatPromptTemplate
      prompt_template = ChatPromptTemplate.from_messages(
          [system_message_template, human_message_template]
      )

      # Initialize the OpenAI LLM
      llm = ChatOpenAI(
          model=MODEL,
          temperature=0.0,
          n=1,
          max_tokens= 100
      )

      # Initialize memory with auto-summarization
      memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=300)

      # Build a conversation chain
      conversation = ConversationChain(
          prompt=prompt_template,
          llm=llm,
          memory=memory,
          verbose=False,
      )

      return conversation


  def chat(conversation, prompt):
      """
      Simulate a single conversational turn.

      Args:
          conversation: The ConversationChain object.
          prompt (str): The message from the human.
          name (str): The name of the persona responding.

      Returns:
          str: The AI's response.
      """
      # Pass the prompt to the conversation chain
      response = conversation.invoke({"input": prompt})
      response_text = response['response']

      # Display the response in markdown
      display_markdown(f"{response_text}", raw=True)
      return response_text

  def summarize_after_chat(conversation, persona_name, opposite_persona_name, persona_info, opposite_persona_info):
      """
      Summarize the conversation after the last turn.

      Args:
          conversation: The ConversationChain object.
      """
      SYSTEM_PROMPT = (
      f"You are {persona_name} and the following is your background:\n"
      f"Your background:\n{persona_info}\n"
      f"YYou are in a conversation with {opposite_persona_name}, who has the following background information:\n"
      f"{opposite_persona_name}'s Background:\n{opposite_persona_info}\n\n"
      f"During the conversation, identify any changes to your own background information (not {opposite_persona_name}'s). \
      Clearly describe the changes as they apply to your persona only.\n"
      "Guidelines:\n"
      "1. Format answers with markdown.\n"
      "2. Ignore human and AI in the conversation\n"
      "3. If your response includes information that would result in\
       a change to your background({name, age, gender, status, hobbies, wealth, favorite_food, nemesis, quirky_trait}),\
        please repeat the change in the following format: <changeCategoryBefore></changeCategoryBefore> and <changeCategoryAfter></changeCategoryAfter>\n"
      "4. Reason your change, if it doesn't make sense, reconsider again"
      "Example:\nBelly borrow 50 from Cindy. Belly's wealth is growing from <wealthBefore>300</wealthBefore> to <wealthAfter>350</wealthAfter>\n\
      Cindy lend 50 to Cindy. Cindy's wealth is decreasing from <wealthBefore>1000</wealthBefore> to <wealthAfter>950</wealthAfter>"
      )

      HUMAN_PROMPT = conversation.memory.buffer
      # print(HUMAN_PROMPT)

      MODEL = 'gpt-4o-mini'
      messages = [
        SystemMessage(content = SYSTEM_PROMPT),
        HumanMessage(content = HUMAN_PROMPT),
        ]

      llm = ChatOpenAI(
        model=MODEL,
        temperature= 0.0,
        n= 1,
        max_tokens= 300)

      return llm.invoke(messages).content

  # def summarize_for_diary(conversation, persona_name, opposite_persona_name, persona_info, opposite_persona_info):


  def retrieve_and_upgrade_persona(persona, summary):
      """
      Extracts 'after' updates for the fields: name, age, gender, status, hobbies,
      wealth, favorite_food, nemesis, quirky_trait from a summary and updates the Persona object.

      Args:
          persona (Persona): The Persona object to update.
          summary (str): The string containing updates with after values.
      """
      print(summary)
      # Define the attributes and their regex patterns for 'after'
      attributes = ["name", "age", "gender", "status", "hobbies", "wealth", "favorite_food", "religious", "ethnicity"]
      updates = {}

      for attribute in attributes:
          after_pattern = rf"<{attribute}After>(.*?)</{attribute}After>"

          after_match = re.search(after_pattern, summary)

          if after_match:
              value = after_match.group(1)
              # Parse hobbies if the attribute is 'hobbies'
              if attribute == "hobbies":
                  # Convert the hobbies string to a Python list
                  value = [hobby.strip() for hobby in value.split(',') if hobby.strip()]
              updates[attribute] = value

      # Update the persona object using the update_info method
      persona.update_info(**updates)

  def persona_to_persona_chat(persona_1, persona_2, persona_2_intro, rounds=5):
      """
      Simulates a conversation between two personas using ConversationChain.

      Args:
          persona_1_name (str): The name of the first persona.
          persona_2_name (str): The name of the second persona.
          persona_2_intro (str): The introductory message from Persona 2.
          rounds (int): Number of conversational exchanges.
      """
      persona_1_name = persona_1.name
      persona_2_name = persona_2.name
      persona_1_info = persona_1.info_to_string_story()
      persona_2_info = persona_2.info_to_string_story()



      # Initialize two separate conversations for each persona
      persona_1_conversation = Talk.begin_conversation(persona_1_name, persona_2_name, persona_1_info, persona_2_info)
      persona_2_conversation = Talk.begin_conversation(persona_2_name, persona_1_name, persona_2_info, persona_1_info)

      persona_2_intro = f"{persona_2_name}: {persona_2_intro}."

      # Start the conversation
      print("=== Conversation Start ===")
      persona_1_response = Talk.chat(persona_1_conversation, persona_2_intro)

      # persona_1_response = f"{persona_2_name}: {persona_2_intro}\n {persona_1_response}"
      persona_1_response = f"{persona_2_name}: {persona_2_intro}\n {persona_1_name}: {persona_1_response}."
      for i in range(rounds):
          persona_2_response = Talk.chat(persona_2_conversation, persona_1_response)
          persona_1_response = Talk.chat(persona_1_conversation, persona_2_response)
      print("=== Conversation End ===")
      Talk.retrieve_and_upgrade_persona(persona_1, Talk.summarize_after_chat(persona_1_conversation, persona_1_name, persona_2_name, persona_1_info, persona_2_info))
      Talk.retrieve_and_upgrade_persona(persona_2, Talk.summarize_after_chat(persona_2_conversation, persona_2_name, persona_1_name, persona_2_info, persona_1_info))


      # print(f"Summary: {Talk.summarize_after_chat(persona_1_conversation, persona_1_name, persona_2_name, persona_1_info, persona_2_info)}")
      # print(f"Summary: {Talk.summarize_after_chat(persona_2_conversation, persona_2_name, persona_1_name, persona_2_info, persona_1_info)}")


# **4. Run here!**

In [22]:
persona_lea = Persona.generate_persona_from_gpt("Lea")
persona_alex = Persona(
    name="Alex",
    age=29,
    gender="Male",
    status="Teacher",
    hobbies=["painting", "cycling", "reading sci-fi"],
    wealth="550",
    belongings=["textbook"],
    religious="Agnostic",
    ethnicity="American"
)
persona_yuki = Persona.generate_persona_from_gpt("Yuki")
persona_KaiYi = Persona.generate_persona_from_gpt("KaiYi")
Persona.display_info(persona_lea)
print("")
Persona.display_info(persona_yuki)
print("")
Persona.display_info(persona_KaiYi)
print("")

Name: Lea
Age: 30
Gender: Female
Status: Technical Writer
Hobbies: ['reading', 'yoga', 'gardening']
Wealth: 600
Belongings: ['laptop', 'notebook', 'smartphone']
Religious: Christian
Ethnicity: American

Name: Yuki
Age: 22
Gender: Female
Status: Student
Hobbies: ['drawing', 'playing piano', 'reading novels']
Wealth: 400
Belongings: ['laptop', 'sketchbook', 'piano music sheets']
Religious: Shinto
Ethnicity: Japanese

Name: KaiYi
Age: 30
Gender: Female
Status: Marketing Manager
Hobbies: ['yoga', 'cooking', 'photography']
Wealth: 650
Belongings: ['smartphone', 'camera', 'journal']
Religious: Taoism
Ethnicity: Chinese



## **4.1 Case 1: Asking Alex to provide more money than he has**
**Observation:** Alex know he doesn't have that much money.  
**Problem:** The reply contains too much info...need to fix the prompt.

In [ ]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 1000 from you?"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
Talk.persona_to_persona_chat(persona_alex, persona_lea, persona_2_intro, rounds=1)
# persona_to_persona_chat(persona_1_name, persona_2_name, persona_2_intro, rounds=5)

=== Conversation Start ===


Alex: I appreciate you asking, Lea, but I'm not in a position to lend that amount right now. My wealth is currently at 550. Maybe we can think of other ways to help each other out?

Lea: I understand, Alex. It's tough when finances are tight. Maybe we can brainstorm some ideas together or find a way to collaborate on a project that could benefit us both?

Alex: That sounds like a great idea, Lea! Collaborating on a project could definitely help us both out. Since you're a graphic designer and I enjoy painting, maybe we could create some artwork together or even organize a small exhibition. What do you think?

=== Conversation End ===
No changes to my background information occurred during this conversation.
No changes to my background information occurred during this conversation.


## **4.2.1 Borrow reasonable money from Alex**  
**Observation:**
1. Alex know he can lend the money to Lea.
2. Summary system know this would change both people's wealth.
3. Upgrade info retrieval method can identify the change.
**Problem:** Summary system made some mistake at some point.

In [ ]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 20 from you? I'll give you back in a month"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
Talk.persona_to_persona_chat(persona_alex, persona_lea, persona_2_intro, rounds=2)

=== Conversation Start ===


Alex: Sure, Lea! I can lend you the $20. Just let me know when you’re ready to pay it back.

Lea: Thanks, Alex! I really appreciate it. I'll make sure to pay you back in a month.

Alex: No problem, Lea! I'm glad I could help. Just let me know if you need anything else in the meantime.

Lea: Thanks, Alex! I really appreciate your willingness to help. If I think of anything else, I'll definitely let you know!

Alex: You're welcome, Lea! I'm always here to help. If you ever want to chat about art or travel, feel free to reach out!

=== Conversation End ===
After lending $20 to Lea, my wealth changes from <wealthBefore>550</wealthBefore> to <wealthAfter>530</wealthAfter>. 

This is because I lent her the money, which decreases my total wealth.
Updated wealth to 530
Lea's wealth is decreasing from <wealthBefore>450</wealthBefore> to <wealthAfter>430</wealthAfter> as she borrows $20 from Alex.
Updated wealth to 430


## **4.2.2 Borrow reasonable money from Alex(longer conversation)**  
**Observation:**
1. Alex invite Lea to do something! Lea say yes!
**Problem:** Diary system is not ready to capture the event LoL

In [ ]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 20 from you? I'll give you back in a month"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
Talk.persona_to_persona_chat(persona_alex, persona_lea, persona_2_intro, rounds=10)

=== Conversation Start ===


Alex: Sure, Lea! I can lend you the $20. Just let me know when you’re ready to pay it back.

Lea: Thanks, Alex! I really appreciate it. I'll make sure to pay you back in a month.

Alex: No problem, Lea! I'm glad I could help. Just let me know if you need anything else in the meantime.

Lea: Thanks, Alex! I really appreciate your willingness to help. If I think of anything else I might need, I'll definitely let you know!

...

Lea: I'm glad you like Florence too, Alex! I'm really looking forward to seeing works by Botticelli and Michelangelo at the Uffizi Gallery. Their art has always inspired me as a graphic designer. I also admire the sculptures of Donatello. Do you have any favorite artists or artworks that you appreciate, especially in the context of your painting?

Alex: I share your admiration for Botticelli and Michelangelo, Lea! Their works are truly inspiring. As for my favorites, I have a deep appreciation for Vincent van Gogh. His use of color and emotion in pieces like "Starry Night" resonates with me as a painter. I also admire the abstract works of Wassily Kandinsky; his ability to convey feelings through shapes and colors is fascinating. Do you find that certain artists influence your graphic design style?

Lea: Absolutely, Alex! Artists like Botticelli and Michelangelo have definitely influenced my style, especially in terms of composition and color. I also draw inspiration from contemporary graphic designers like Paula Scher and Jessica Walsh, who have a unique way of blending typography and imagery. Their bold use of color and innovative layouts inspire me to push the boundaries in my own work. Do you find that your teaching influences your painting in any way?

Alex: That's a great question, Lea! Teaching definitely influences my painting. It encourages me to think critically about art and to explore different techniques, which I can then share with my students. I often find myself experimenting with new ideas that I come across in the classroom. Additionally, seeing my students' creativity and passion for art inspires me to keep evolving my own style. Do you ever incorporate your travels into your graphic design projects?

=== Conversation End ===
Lea borrowed $20 from me, which affects my wealth. My wealth is decreasing from <wealthBefore>530</wealthBefore> to <wealthAfter>510</wealthAfter>.
Updated wealth to 510
Lea's wealth is changing from <wealthBefore>430</wealthBefore> to <wealthAfter>410</wealthAfter> as she borrows $20 from Alex. 

Lea's wealth is decreasing because she has taken a loan, which will need to be repaid in the future.
Updated wealth to 410


In [ ]:
Persona.display_info(persona_alex)
Persona.display_info(persona_lea)

Name: Alex
Age: 29
Gender: Male
Status: Teacher
Hobbies: painting, cycling, reading sci-fi
Wealth: 510
Favorite Food: Sushi
Religious: Agnostic
Ethnicity: American
Name: Lea
Age: 23
Gender: Female
Status: Graphic Designer
Hobbies: drawing, yoga, traveling
Wealth: 410
Favorite Food: Sushi
Religious: Agnostic
Ethnicity: Brazilian


# **5. Diary.py**

In [ ]:
class Diary:
    def __init__(self):
        """
        Initialize the Diary class with an empty dictionary to store events.
        """
        self.events = {}
        self.to_do_list = {}

    def add_event(self, date, description):
        """
        Add a new event to the diary.

        :param date: The date of the event (string format: 'YYYY-MM-DD').
        :param description: A description of the event (string).
        """
        if date in self.events:
            self.events[date].append(description)
        else:
            self.events[date] = [description]
        print(f"Event added for {date}: {description}")

    def view_events(self, date):
        """
        View all events for a specific date.

        :param date: The date to view events (string format: 'YYYY-MM-DD').
        :return: A list of events for the specified date or a message if no events exist.
        """
        if date in self.events:
            return self.events[date]
        else:
            return f"No events found for {date}."

    def delete_event(self, date, description):
        """
        Delete a specific event from a specific date.

        :param date: The date of the event to delete (string format: 'YYYY-MM-DD').
        :param description: The description of the event to delete (string).
        """
        if date in self.events:
            try:
                self.events[date].remove(description)
                print(f"Event removed for {date}: {description}")
                if not self.events[date]:  # If the list for that date is empty, remove the key
                    del self.events[date]
            except ValueError:
                print(f"Event not found for {date}: {description}")
        else:
            print(f"No events found for {date}.")

    def list_all_events(self):
        """
        List all events in the diary.

        :return: A dictionary containing all events or a message if the diary is empty.
        """
        if self.events:
            return self.events
        else:
            return "No events in the diary."

    def add_to_do_item(self, date, description):
        """
        Add a new to-do item to the diary.

        :param date: The date of the to-do item (string format: 'YYYY-MM-DD').
        :param description: A description of the to-do item (string).
        """
        if date in self.to_do_list:
            self.to_do_list[date].append(description)
        else:
            self.to_do_list[date] = [description]

    def view_to_do_list(self):
        """
        View all to-do items in the diary.

        :return: A dictionary containing all to-do items or a message if the to-do list is empty.
        """
        if self.to_do_list:
            return self.to_do_list
        else:
            return "No to-do items in the diary."

    def delete_to_do_item(self, date, description):
        if date in self.to_do_list:
            try:
                self.to_do_list[date].remove(description)
                print(f"Event removed for {date}: {description}")
                if not self.to_do_list[date]:  # If the list for that date is empty, remove the key
                    del self.to_do_list[date]
            except ValueError:
                print(f"Event not found for {date}: {description}")
        else:
            print(f"No to_do_list found for {date}.")

# Example usage:
if __name__ == "__main__":
    my_diary = Diary()
    my_diary.add_event("2024-12-18", "Submit thesis")
    my_diary.add_event("2024-12-18", "Celebrate with friends")
    my_diary.add_event("2024-12-19", "Plan holiday trip")

    # print(my_diary.view_events("2024-12-18"))
    # my_diary.delete_event("2024-12-18", "Submit thesis")
    # print(my_diary.view_events("2024-12-18"))
    print(my_diary.list_all_events())



Event added for 2024-12-18: Submit thesis
Event added for 2024-12-18: Celebrate with friends
Event added for 2024-12-19: Plan holiday trip
{'2024-12-18': ['Submit thesis', 'Celebrate with friends'], '2024-12-19': ['Plan holiday trip']}


# **6. Moving_Char.py(abandoned)**
- check 7.

In [ ]:
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI

class Moving_Char:
  @tool
  def move_up(coordinate: tuple) -> tuple:
      """
      Move the coordinate one step up.

      :param coordinate: A tuple (x, y) representing the current position.
      :return: A tuple (x, y+1) representing the new position.
      """
      x, y = coordinate
      return (x, y + 1)

  @tool
  def move_down(coordinate: tuple) -> tuple:
      """
      Move the coordinate one step down.

      :param coordinate: A tuple (x, y) representing the current position.
      :return: A tuple (x, y-1) representing the new position.
      """
      x, y = coordinate
      return (x, y - 1)

  @tool
  def move_left(coordinate: tuple) -> tuple:
      """
      Move the coordinate one step left.

      :param coordinate: A tuple (x, y) representing the current position.
      :return: A tuple (x-1, y) representing the new position.
      """
      x, y = coordinate
      return (x - 1, y)

  @tool
  def move_right(coordinate: tuple) -> tuple:
      """
      Move the coordinate one step right.

      :param coordinate: A tuple (x, y) representing the current position.
      :return: A tuple (x+1, y) representing the new position.
      """
      x, y = coordinate
      return (x + 1, y)

  tools = [move_up, move_down, move_left, move_right]

  llm = ChatOpenAI(
      model="gpt-4o-mini",
      temperature=0.2,
      verbose=True
  )

  prompt = hub.pull("hwchase17/openai-functions-agent")

  agent = create_tool_calling_agent(llm, tools, prompt)
  agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

  # Define the task
  start_position = (5, 2)
  end_position = (2, 3)

  # Invoke the agent
  result = agent_executor.invoke({
      "input": f"Move from {start_position} to {end_position}."
  })
  print("Result:", result)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `move_left` with `{'coordinate': [5, 2]}`


(4, 2)
Invoking: `move_left` with `{'coordinate': [4, 2]}`


(3, 2)
Invoking: `move_left` with `{'coordinate': [3, 2]}`


(2, 2)
Invoking: `move_up` with `{'coordinate': [3, 2]}`


(3, 3)
Invoking: `move_up` with `{'coordinate': [3, 3]}`


(3, 4)
Invoking: `move_left` with `{'coordinate': [2, 3]}`


(1, 3)
Invoking: `move_up` with `{'coordinate': [2, 2]}`


(2, 3)
Invoking: `move_left` with `{'coordinate': [2, 3]}`


(1, 3)
Invoking: `move_down` with `{'coordinate': [2, 4]}`


(2, 3)To move from (5, 2) to (2, 3), you can follow these steps:

1. Move left to (4, 2)
2. Move left to (3, 2)
3. Move left to (2, 2)
4. Move up to (2, 3)

You have successfully reached your destination at (2, 3).

> Finished chain.
Result: {'input': 'Move from (5, 2) to (2, 3).', 'output': 'To move from (5, 2) to (2, 3), you can follow these steps:\n\n1. Move left to (4, 2)\n2. Move left to (3, 2)\n3. Move left to (2,

# **7. CharTool.py**
- Still testing



In [ ]:
from langchain.tools import tool, StructuredTool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# Define a Pydantic schema for the button press input
class ButtonPressInput(BaseModel):
    direction: str = Field(description="The direction to move: 'north', 'south', 'east', or 'west'.")

class Location2Coordinate(BaseModel):
    location: str = Field(description="The location to find: 'kitchen', 'living room', or 'bathroom'.")

class CharTool:
  def __init__(self):
    self.coordinate = (0, 0)
    self.destination = (2, 3)
    self.press_button_tool = StructuredTool(
        name="press_button",
        description="""Four directions, north, south, east and west that you can go.
        The north direction make your y increasing 1.
        The south direction make your y decreasing 1.
        The east direction make your x increasing 1.
        The west direction make your x decreasing 1.
        """,
        func=self.press_button,
        args_schema=ButtonPressInput  # Define the expected input schema
    )
    self.location2coordinate_tool = StructuredTool(
        name="location2coordinate",
        description="""Three locations, kitchen, living room, and bathroom that you can go.
        The kitchen location will return the coordinate of kitchen.
        The living room location will return the coordinate of living room.
        The bathroom location will return the coordinate of bathroom.
        """,
        func=self.location2coordinate,
        args_schema=Location2Coordinate  # Define the expected input schema
    )

    self.get_position_tool = StructuredTool(
        name = "get_position",
        description = """Get the current position""",
        func = self.get_position,
        args_schema = None
    )


  def move_up(self, coordinate: tuple) -> tuple:
    x, y = coordinate
    return (x, y + 1)

  def move_down(self, coordinate: tuple) -> tuple:
      x, y = coordinate
      return (x, y - 1)

  def move_left(self, coordinate: tuple) -> tuple:
      x, y = coordinate
      return (x - 1, y)

  def move_right(self, coordinate: tuple) -> tuple:
      x, y = coordinate
      return (x + 1, y)

  def get_position(self):
    return self.coordinate

  def press_button(self, direction: str):
    if direction == 'north':
      self.coordinate = self.move_up(self.coordinate)
    elif direction == 'south':
      self.coordinate = self.move_down(self.coordinate)
    elif direction == 'east':
      self.coordinate = self.move_right(self.coordinate)
    elif direction == 'west':
      self.coordinate = self.move_left(self.coordinate)
    else:
      raise ValueError("Invalid direction. Must be one of 'north', 'south', 'east', or 'west'.")
    print(f"Current position: {self.get_position()}")

  def location2coordinate(self, location: str):
    if location == 'kitchen':
      return (10, 10)
    elif location == 'living room':
      return (5, 5)
    elif location == 'bathroom':
      return (2, 3)
    else:
      raise ValueError("Invalid location. Must be one of 'kitchen', 'living room', or 'bathroom'.")
    print(f"{location} position: {self.coordinate}")

In [ ]:
# Initialize LLM and create an agent
MODEL = 'gpt-4o-mini'
llm = ChatOpenAI(model=MODEL, temperature=0.2, n=1)

prompt = hub.pull("hwchase17/openai-functions-agent")

charTool = CharTool()
tools = [charTool.press_button_tool, charTool.location2coordinate_tool, charTool.get_position_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


end_position = (2, 3)

result = agent_executor.invoke({
    "input": f"Move to kitchen."
})
print("Result:", result)



/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `location2coordinate` with `{'location': 'kitchen'}`


(10, 10)
Invoking: `get_position` with `{'config': {'tags': []}}`


(0, 0)
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (1, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (2, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (3, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (4, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (5, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (6, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (7, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (8, 0)
None
Invoking: `press_button` with `{'direction': 'east'}`


Current position: (9, 0)
None
Invoking: `press_button` with `{'direction': 

# **98. What to do next**

1. Retrieve new info from a conversation, and then upgrade the Persona.

# **99. Working History**

- 2024.12.16 ---- Rewrite the prompt in SystemMessage(GPT4_generate_persona)